# Apply trace metal concentrations to river classification

river categories:
1. Glaciers
2. Continental
3. Other

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean

%matplotlib inline

In [2]:
rclass = nc.Dataset('/ocean/brogalla/GEOTRACES/data/river_class-202005.nc','r')
river_class = rclass.variables['rclass']
river_class = np.array(river_class)

Based on Manuel's projection ranges for Mn:
- northern rivers measured range:     0-10   nmol/kg
- northern rivers projected range:    0-100  nmol/kg
- continental rivers measured range:  15-55  nmol/kg
- continental rivers projected range: 15-750 nmol/kg

Based on Manuel's projection ranges for Pb:
- northern rivers measured range:     0-8   pmol/kg
- northern rivers projected range:    0-15  pmol/kg
- continental rivers measured range:  14-73  pmol/kg
- continental rivers projected range: 14-152 pmol/kg

Manganese:

In [3]:
pMn_rivers = np.zeros(river_class.shape)
pMn_rivers[river_class==1.0] = 151 # glaciers
pMn_rivers[river_class==2.0] = 13  # continental
pMn_rivers[river_class==3.0] = 2   # other
pMn_rivers[river_class==4.0] = 2   # Gulf of Boothia

In [4]:
dMn_rivers = np.zeros(river_class.shape)
dMn_rivers[river_class==1.0] = 164 # glaciers
dMn_rivers[river_class==2.0] = 750  # continental
dMn_rivers[river_class==3.0] = 100   # other
dMn_rivers[river_class==4.0] = 5   # Gulf of Boothia

Lead:

In [5]:
pPb_rivers = np.zeros(river_class.shape)
pPb_rivers[river_class==1.0] = 5 # glaciers
pPb_rivers[river_class==2.0] = 1  # continental
pPb_rivers[river_class==3.0] = 0.2   # other
pPb_rivers[river_class==4.0] = 0.2   # other

In [6]:
dPb_rivers = np.zeros(river_class.shape)
dPb_rivers[river_class==1.0] = 35 # glaciers
dPb_rivers[river_class==2.0] = 152  # continental
dPb_rivers[river_class==3.0] = 15   # other
dPb_rivers[river_class==4.0] = 15   # other

#### Convert units to mol/L:

In [7]:
dMn_rivers = dMn_rivers*10**(-9)      # nM --> 10^-3 mol/L
pMn_rivers = pMn_rivers*(1/54.938049)*10**(-6) # micro g/L --> 1.82*10^-2 mol/L

In [8]:
dPb_rivers = dPb_rivers*10**(-12)           # pM --> 10^-3 mol/L
pPb_rivers = pPb_rivers*(1/207.2)*10**(-6) # micro g/L --> 1.82*10^-2 mol/L

# Write to NetCDF file

In [9]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/runoff/'+\
               'ANHA12_runoff_monthly_combined_Dai_Trenberth_Bamber_y2002.nc','r')
lon_rf = np.array(c.variables['nav_lon'])
lat_rf = np.array(c.variables['nav_lat'])

In [10]:
ncd = nc.Dataset('/ocean/brogalla/GEOTRACES/data/river-forcing-20200807-projected.nc', 'w', zlib=True)

ncd.createDimension('x',len(c.dimensions['x']))
ncd.createDimension('y',len(c.dimensions['y']))

<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 2400

In [11]:
# variables
nav_lat = ncd.createVariable('nav_lat', 'float32', ('y','x'))
nav_lat.long_name = 'Latitude'
nav_lat.units = 'degrees_north'
nav_lat[:] = lat_rf

nav_lon = ncd.createVariable('nav_lon', 'float32', ('y','x'))
nav_lon.long_name = 'Longitude'
nav_lon.units = 'degrees_east'
nav_lon[:] = lon_rf

pmn_rivers = ncd.createVariable('pmn_rivers', 'float32', ('y','x'))
pmn_rivers.units = 'mol/L'
pmn_rivers.long_name = 'Oxidised Mn concentration'  
pmn_rivers.coordinates = 'nav_lon nav_lat'
pmn_rivers[:] = pMn_rivers

dmn_rivers = ncd.createVariable('dmn_rivers', 'float32', ('y','x'))
dmn_rivers.units = 'mol/L'
dmn_rivers.long_name = 'Dissolved Mn concentration'  
dmn_rivers.coordinates = 'nav_lon nav_lat'
dmn_rivers[:] = dMn_rivers

ppb_rivers = ncd.createVariable('ppb_rivers', 'float32', ('y','x'))
ppb_rivers.units = 'mol/L'
ppb_rivers.long_name = 'Particulate Pb concentration'  
ppb_rivers.coordinates = 'nav_lon nav_lat'
ppb_rivers[:] = pPb_rivers

dpb_rivers = ncd.createVariable('dpb_rivers', 'float32', ('y','x'))
dpb_rivers.units = 'mol/L'
dpb_rivers.long_name = 'Dissolved Pb concentration'  
dpb_rivers.coordinates = 'nav_lon nav_lat'
dpb_rivers[:] = dPb_rivers

In [12]:
ncd.close()